In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset

In [2]:
df_title= pd.read_csv('Movie_Id_Titles')
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  1682 non-null   int64 
 1   title    1682 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [3]:
movielens = Dataset.load_builtin('ml-100k')
movielens = pd.DataFrame(movielens.raw_ratings, columns = ['user_id', 'item_id', 'rating', 'timestamps'])
movielens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     100000 non-null  object 
 1   item_id     100000 non-null  object 
 2   rating      100000 non-null  float64
 3   timestamps  100000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [4]:
movielens[['user_id', 'item_id', 'rating']] = movielens[['user_id', 'item_id', 'rating']].astype('int64')
movielens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   item_id     100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   timestamps  100000 non-null  object
dtypes: int64(3), object(1)
memory usage: 3.1+ MB


In [5]:
movielens = pd.merge(movielens, df_title, on = 'item_id')

In [6]:
movielens.head()

,user_id,item_id,rating,timestamps,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [7]:
movielens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   item_id     100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   timestamps  100000 non-null  object
 4   title       100000 non-null  object
dtypes: int64(3), object(2)
memory usage: 4.6+ MB


In [8]:
movielens[movielens['item_id'] == 50]

,user_id,item_id,rating,timestamps,title
50711,290,50,5,880473582,Star Wars (1977)
50712,79,50,4,891271545,Star Wars (1977)
50713,2,50,5,888552084,Star Wars (1977)
50714,8,50,5,879362124,Star Wars (1977)
50715,274,50,5,878944679,Star Wars (1977)
...,...,...,...,...,...
51289,749,50,5,878846978,Star Wars (1977)
51290,886,50,5,876031501,Star Wars (1977)
51291,739,50,4,886958895,Star Wars (1977)
51292,903,50,5,891031329,Star Wars (1977)


In [9]:
movielens.corr('pearson')

,user_id,item_id,rating
user_id,1.000000,0.010377,-0.009371
item_id,0.010377,1.000000,-0.189124
rating,-0.009371,-0.189124,1.000000


In [24]:
moviematrix = movielens.pivot_table(index = 'item_id', columns = 'user_id', values = 'rating')

In [25]:
moviematrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
moviematrix[290]

item_id
1       5.0
2       NaN
3       NaN
4       NaN
5       NaN
       ... 
1678    NaN
1679    NaN
1680    NaN
1681    NaN
1682    NaN
Name: 290, Length: 1682, dtype: float64

In [27]:
cor = moviematrix.corr(method = 'pearson')

In [28]:
cor

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,1.608412e-01,0.112780,0.500000,0.420809,0.287159,0.258137,0.692086,-0.102062,-0.092344,...,0.061695,-2.602417e-01,0.383733,2.899974e-02,0.326744,5.343904e-01,0.263289,0.205616,-0.180784,0.067549
2,0.160841,1.000000e+00,0.067420,0.148522,0.327327,0.446269,0.643675,0.585491,0.242536,0.668145,...,0.021007,-2.711631e-01,0.214017,5.616449e-01,0.331587,-7.671236e-18,-0.011682,-0.062017,0.085960,0.479702
3,0.112780,6.741999e-02,1.000000,-0.262600,NaN,-0.109109,0.064803,0.291937,NaN,0.311086,...,NaN,NaN,-0.045162,-5.233642e-17,-0.137523,NaN,-0.104678,1.000000,-0.011792,NaN
4,0.500000,1.485221e-01,-0.262600,1.000000,1.000000,-0.581318,-0.266632,0.642938,NaN,-0.301511,...,0.500000,NaN,-0.203653,NaN,0.375000,NaN,0.850992,1.000000,0.412568,NaN
5,0.420809,3.273268e-01,NaN,1.000000,1.000000,0.241817,0.175630,0.537400,0.577350,0.087343,...,0.229532,-5.000000e-01,0.439286,6.085806e-01,0.484211,8.807048e-01,0.027038,0.468521,0.318163,0.346234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.534390,-7.671236e-18,NaN,NaN,0.880705,0.206315,0.142404,-0.333333,NaN,0.316228,...,0.374351,-3.305898e-02,0.471172,-2.758386e-01,-0.073374,1.000000e+00,-0.534522,-0.131306,-0.500000,-0.187317
940,0.263289,-1.168173e-02,-0.104678,0.850992,0.027038,-0.024419,0.000931,0.320487,0.171499,0.158976,...,-0.125059,4.352858e-01,-0.338327,-1.486075e-01,0.110022,-5.345225e-01,1.000000,0.632746,-0.022813,0.332497
941,0.205616,-6.201737e-02,1.000000,1.000000,0.468521,0.399186,0.000000,0.166667,1.000000,0.420084,...,-0.500000,-2.355139e-16,0.273060,3.929526e-01,-0.214147,-1.313064e-01,0.632746,1.000000,-0.577350,-0.395285


In [29]:
top30_users = cor[290].sort_values(ascending = False).head(31)
top30_users.drop(290, inplace = True)
top30_users

user_id
451    1.000000
724    1.000000
651    1.000000
131    1.000000
863    1.000000
252    1.000000
520    1.000000
578    1.000000
362    1.000000
750    0.944911
209    0.942809
489    0.927173
462    0.912871
797    0.912871
799    0.904534
814    0.870388
444    0.866025
482    0.866025
134    0.866025
423    0.852803
742    0.840152
3      0.818182
310    0.788342
441    0.787296
590    0.759257
598    0.755929
212    0.752549
367    0.750000
104    0.748798
359    0.745957
Name: 290, dtype: float64

In [30]:
neighbors = moviematrix[top30_users.index.tolist()]
neighbors.head()

user_id,451,724,651,131,863,252,520,578,362,750,...,742,3,310,441,590,598,212,367,104,359
item_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


In [31]:
neighbors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1682 entries, 1 to 1682
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   451     98 non-null     float64
 1   724     85 non-null     float64
 2   651     21 non-null     float64
 3   131     30 non-null     float64
 4   863     107 non-null    float64
 5   252     21 non-null     float64
 6   520     23 non-null     float64
 7   578     24 non-null     float64
 8   362     25 non-null     float64
 9   750     33 non-null     float64
 10  209     33 non-null     float64
 11  489     109 non-null    float64
 12  462     33 non-null     float64
 13  797     26 non-null     float64
 14  799     24 non-null     float64
 15  814     35 non-null     float64
 16  444     24 non-null     float64
 17  482     26 non-null     float64
 18  134     25 non-null     float64
 19  423     64 non-null     float64
 20  742     26 non-null     float64
 21  3       54 non-null     float64
 22  

In [32]:
len(neighbors)

1682

# Lets figure out how many movies user ID 451 and 290 have in common (movies that they both rated)

In [125]:
a = movielens[movielens['user_id']==1].sort_values(by = 'item_id', ascending = True)

In [126]:
b = movielens[movielens['user_id']==290].sort_values(by = 'item_id', ascending = True)

In [166]:
a1 = a['item_id'].tolist()
b1 = b['item_id'].tolist()
b1

[1,
 15,
 21,
 22,
 28,
 31,
 43,
 49,
 50,
 54,
 62,
 64,
 66,
 69,
 71,
 82,
 88,
 89,
 91,
 95,
 97,
 98,
 99,
 102,
 105,
 109,
 117,
 118,
 120,
 121,
 125,
 132,
 133,
 135,
 136,
 139,
 141,
 143,
 144,
 151,
 153,
 158,
 161,
 162,
 164,
 167,
 168,
 172,
 174,
 176,
 180,
 181,
 183,
 191,
 193,
 196,
 199,
 202,
 204,
 205,
 208,
 210,
 211,
 216,
 218,
 222,
 227,
 228,
 229,
 230,
 234,
 235,
 239,
 243,
 252,
 257,
 265,
 271,
 274,
 318,
 323,
 357,
 378,
 380,
 385,
 393,
 402,
 403,
 404,
 405,
 418,
 419,
 423,
 429,
 432,
 434,
 435,
 436,
 449,
 450,
 465,
 472,
 473,
 474,
 476,
 483,
 484,
 496,
 498,
 515,
 520,
 523,
 527,
 546,
 550,
 566,
 568,
 588,
 596,
 622,
 625,
 629,
 650,
 651,
 683,
 685,
 692,
 699,
 720,
 732,
 739,
 742,
 755,
 809,
 818,
 825,
 826,
 832,
 926,
 930,
 993,
 1013,
 1028,
 1035,
 1047,
 1060,
 1079,
 1091,
 1285,
 1336]

In [128]:
intersection = set(b1).intersection(a1)
intersection

{1,
 15,
 21,
 22,
 28,
 31,
 43,
 49,
 50,
 54,
 62,
 64,
 66,
 69,
 71,
 82,
 88,
 89,
 91,
 95,
 97,
 98,
 99,
 102,
 105,
 109,
 117,
 118,
 120,
 121,
 125,
 132,
 133,
 135,
 136,
 139,
 141,
 143,
 144,
 151,
 153,
 158,
 161,
 162,
 164,
 167,
 168,
 172,
 174,
 176,
 180,
 181,
 183,
 191,
 193,
 196,
 199,
 202,
 204,
 205,
 208,
 210,
 211,
 216,
 218,
 222,
 227,
 228,
 229,
 230,
 234,
 235,
 239,
 243,
 252,
 257,
 265,
 271}

## It turns out they both rated only 3 movies, which are movie_id 243, 323, 683

In [ ]:
print(a[a['item_id'] == 243][['rating','item_id']],a[a['item_id'] == 323][['rating','item_id']], a[a['item_id'] == 683][['rating','item_id']])

In [ ]:
print(b[b['item_id'] == 243][['rating','item_id']],b[b['item_id'] == 323][['rating','item_id']], b[b['item_id'] == 683][['rating','item_id']])

## If you manually calculate the pearson correlation, then you will get 1 as a result

In [ ]:
import math

pearson_corr_nominator = (3-8/3)*(4-3)+(3-8/3)*(4-3)+(2-8/3)*(1-3)
sum_square_x = (3-8/3)**2 + (3-8/3)**2 + (2-8/3)**2
sum_square_y = (4-3)**2 + (4-3)**2 + (1-3)**2

sqrt = math.sqrt(sum_square_x*sum_square_y)

cor = pearson_corr_nominator/sqrt
cor

## Defining a function that users that have rated at least 20 items as the user id 290


In [89]:
column_names = moviematrix.columns.tolist()
l = []
for user_id in column_names:
    #for ind in moviematrix.index():
    l.append(moviematrix[user_id].tolist())

for i in range(len(moviematrix)):
    for ind in moviematrix.index:
        if l[i][ind] != np.nan:
            
            


nan
nan
nan
nan
nan
2.0
4.0
4.0
nan
nan
4.0
2.0
5.0
3.0
nan
nan
nan
4.0
nan
3.0
3.0
4.0
nan
nan
nan
nan
2.0
nan
nan
nan
4.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
3.0
nan
nan
4.0
nan
nan
nan
nan
nan
4.0
nan
nan
5.0
nan
nan
nan
nan
4.0
nan
nan
nan
nan
3.0
3.0
4.0
nan
nan
nan
nan
nan
nan
nan
3.0
nan
4.0
nan
nan
nan
nan
3.0
4.0
nan
4.0
nan
nan
nan
nan
nan
2.0
nan
nan
5.0
nan
5.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.0
nan
nan
nan
nan
nan
2.0
nan
nan
nan
nan
nan
nan
5.0
3.0
nan
5.0
nan
nan
nan
5.0
5.0
4.0
5.0
5.0
5.0
5.0
nan
nan
nan
nan
nan
2.0
nan
nan
nan
nan
nan
nan
nan
3.0
nan
4.0
3.0
nan
3.0
nan
nan
nan
nan
nan
nan
nan
nan
5.0
4.0
nan
4.0
4.0
4.0
nan
nan
5.0
4.0
4.0
nan
4.0
4.0
nan
4.0
nan
4.0
4.0
nan
5.0
4.0
4.0
3.0
3.0
nan
4.0
4.0
3.0
4.0
4.0
nan
5.0
nan
4.0
3.0
nan
3.0
3.0
3.0
3.0
nan
nan
4.0
4.0
nan
5.0
nan
4.0
nan
nan
5.0
nan
nan
nan
nan
4.0
nan
4.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.0
5.0
nan
nan
nan
4.0
nan
nan
nan
3.0
nan
3.0
nan
nan
nan


IndexError: list index out of range

In [62]:
moviematrix.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
           dtype='int64', name='item_id', length=1682)

In [72]:
l[0][:10]

item_id
1     NaN
2     5.0
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     3.0
10    NaN
Name: 943, dtype: float64

In [85]:
moviematrix.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.0,NaN,NaN,NaN,NaN,2.0,5.0,3.0,4.0,4.0,...,NaN,NaN,4.0,NaN,4.0,NaN,4.0,4.0,NaN,NaN
8,1.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
9,5.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,4.0,...,NaN,1.0,4.0,5.0,3.0,5.0,3.0,NaN,NaN,3.0


In [40]:
moviematrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1682 entries, 1 to 1682
Columns: 943 entries, 1 to 943
dtypes: float64(943)
memory usage: 12.1 MB


In [91]:
moviematrix[290]

item_id
1       5.0
2       NaN
3       NaN
4       NaN
5       NaN
       ... 
1678    NaN
1679    NaN
1680    NaN
1681    NaN
1682    NaN
Name: 290, Length: 1682, dtype: float64

In [100]:
#c = moviematrix[moviematrix['user_id']==290]
moviematrix[290]

item_id
1       5.0
2       NaN
3       NaN
4       NaN
5       NaN
       ... 
1678    NaN
1679    NaN
1680    NaN
1681    NaN
1682    NaN
Name: 290, Length: 1682, dtype: float64

In [156]:
d = [] # list of all users who rated a list of items, contains many list from index 0-942
for user_id in column_names: # user_ids 
    c = movielens[movielens['user_id']== user_id]['item_id'].tolist() # list of all movies(item_ids) that each user rated
    d.append(c)

e= [] # list of users who rated the same 40 items as user 290 
for i in range(943):
    if len(set(b1).intersection(d[i])) >40: #  
        e.append(i+1) # since user id starts at 1

e

[1,
 5,
 6,
 7,
 10,
 11,
 13,
 16,
 18,
 22,
 23,
 38,
 42,
 43,
 44,
 49,
 56,
 59,
 60,
 62,
 64,
 70,
 72,
 82,
 83,
 85,
 87,
 90,
 91,
 92,
 94,
 95,
 102,
 109,
 119,
 125,
 128,
 130,
 144,
 145,
 151,
 158,
 174,
 178,
 184,
 188,
 189,
 194,
 198,
 200,
 201,
 207,
 210,
 213,
 215,
 216,
 222,
 230,
 233,
 234,
 236,
 244,
 246,
 249,
 254,
 256,
 262,
 263,
 267,
 268,
 269,
 271,
 275,
 276,
 279,
 280,
 286,
 290,
 291,
 293,
 295,
 297,
 298,
 299,
 301,
 303,
 305,
 307,
 308,
 311,
 312,
 313,
 314,
 318,
 320,
 325,
 326,
 327,
 328,
 330,
 332,
 334,
 339,
 343,
 344,
 345,
 346,
 347,
 354,
 363,
 373,
 374,
 378,
 379,
 380,
 385,
 387,
 389,
 393,
 394,
 398,
 399,
 401,
 405,
 406,
 407,
 416,
 417,
 425,
 429,
 435,
 437,
 442,
 447,
 450,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 468,
 472,
 474,
 479,
 484,
 487,
 488,
 493,
 495,
 496,
 497,
 500,
 503,
 504,
 505,
 506,
 508,
 514,
 521,
 524,
 532,
 533,
 535,
 536,
 537,
 538,
 541,
 542,
 543,
 545

In [165]:
# lets test if we got the right users, so we are going to investigate if user 1 has at least 40 items in common with user 290
# the answer is yesssss

c = movielens[movielens['user_id']== 44]['item_id'].tolist()
h = set(b1).intersection(c)
len(h)

74

In [172]:
len(movielens[movielens['user_id']== 44][['item_id', 'user_id']].sort_values(by = 'item_id',ascending = True)) # rated 151 items and 74 in common with user 290

151

In [175]:
intersect = set(b1).intersection(c)


In [189]:
# lets see which ones they have in common
m = movielens[movielens['user_id']== 44][['item_id', 'user_id']].sort_values(by = 'item_id',ascending = True)

In [190]:
b.merge(m, on = 'item_id')


,user_id_x,item_id,rating,timestamps,title,user_id_y
0,290,1,5,880474327,Toy Story (1995),44
1,290,15,4,880474494,Mr. Holland's Opus (1995),44
2,290,21,3,880475695,Muppet Treasure Island (1996),44
3,290,22,5,880473942,Braveheart (1995),44
4,290,31,4,880475032,Crimson Tide (1995),44
...,...,...,...,...,...,...
69,290,523,3,880473735,Cool Hand Luke (1967),44
70,290,588,4,880474652,Beauty and the Beast (1991),44
71,290,625,4,880475782,"Sword in the Stone, The (1963)",44
72,290,692,5,880474293,"American President, The (1995)",44


In [153]:
#name of the user_id
#column_names
c = movielens[movielens['user_id']== 451]['item_id'].tolist()   # list of all movies(item_ids) that user_id rated

In [154]:
c

[242,
 302,
 304,
 327,
 332,
 322,
 288,
 690,
 294,
 328,
 307,
 264,
 245,
 268,
 323,
 258,
 360,
 748,
 688,
 334,
 872,
 303,
 749,
 1295,
 321,
 984,
 879,
 678,
 1393,
 333,
 286,
 325,
 259,
 938,
 305,
 289,
 873,
 306,
 680,
 301,
 292,
 948,
 1022,
 877,
 269,
 324,
 937,
 319,
 875,
 683,
 995,
 299,
 682,
 331,
 335,
 263,
 300,
 243,
 308,
 886,
 337,
 687,
 874,
 457,
 260,
 990,
 882,
 883,
 880,
 326,
 262,
 329,
 988,
 1394,
 330,
 1025,
 266,
 876,
 887,
 336,
 270,
 878,
 359,
 884,
 261,
 1265,
 358,
 991,
 681,
 989,
 881,
 1296,
 885,
 1392,
 1395,
 1038,
 1280,
 1026]